In [1]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
female_formal_1 = pl.read_parquet('data/female_formal_1.parquet')
female_formal_2 = pl.read_parquet('data/female_formal_2.parquet')

female_informal_1 = pl.read_parquet('data/female_informal_1.parquet')
female_informal_2 = pl.read_parquet('data/female_informal_2.parquet')

male_formal_1 = pl.read_parquet('data/male_formal_1.parquet')
male_formal_2 = pl.read_parquet('data/male_formal_2.parquet')

male_informal_1 = pl.read_parquet('data/male_informal_1.parquet')
male_informal_2 = pl.read_parquet('data/male_informal_2.parquet')

In [3]:
n_au = 27 # Nr of action units

# Reference ids of the action units
au_ref = [1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22,
          23, 24, 25, 26, 27, 32, 38, 39]

def extract_au(df, threshold=0.9):

    features = df.select(
        [   
            'time',
            'frame',
            'face_prob',
            pl.lit([list(range(n_au))]).alias("face_au"),
            pl.col("face_aus").alias("value").list.take(list(range(n_au)))
        ]
    ).filter((pl.col("value").is_not_null()) & 
             (pl.col("face_prob") > threshold) &
             (pl.col("time") <= 50)
             ).explode(["face_au", "value"])

    features = features.to_pandas().drop(columns=['face_prob'])
    features['face_au'] = features['face_au'].apply(lambda x: au_ref[x])
    return features
    

# female
female_formal_1_au = extract_au(female_formal_1)
female_formal_2_au = extract_au(female_formal_2)

female_informal_1_au = extract_au(female_informal_1)
female_informal_2_au = extract_au(female_informal_2)

female_formal_au = pd.concat([female_formal_1_au, female_formal_2_au], keys=['Female Formal 1', ' Female Formal 2'])
female_informal_au = pd.concat([female_informal_1_au, female_informal_2_au], keys=['Female Inormal 1', ' Female Inormal 2'])

female = pd.concat([female_formal_au, female_informal_au])

# male
male_formal_1_au = extract_au(male_formal_1)
male_formal_2_au = extract_au(male_formal_2)

male_informal_1_au = extract_au(male_informal_1)
male_informal_2_au = extract_au(male_informal_2)

male_formal_au = pd.concat([male_formal_1_au, male_formal_2_au], keys=['Male Formal 1', ' Male Formal 2'])
male_informal_au = pd.concat([male_informal_1_au, male_informal_2_au], keys=['Male Informal 1', ' Male Informal 2'])

male = pd.concat([male_formal_au, male_informal_au])

# formal
formal = pd.concat([male_formal_au, female_formal_au])

# informal
informal = pd.concat([male_informal_au, female_informal_au])

# print out the length of male, female, formal and informal
print(f"Male: {male.shape[0]}")
print(f"Female: {female.shape[0]}")
print(f"Formal: {formal.shape[0]}")
print(f"Informal: {informal.shape[0]}")

Male: 25434
Female: 24921
Formal: 25677
Informal: 24678


/var/folders/jh/mjfcxr_509gc9z3gd91thc0w0000gn/T/ipykernel_44678/2250504916.py:14: DeprecationWarning: Behavior for `lit` will change for sequence inputs. The result will change to be a literal of type List. To retain the old behavior, pass a Series instead, e.g. `Series(sequence)`.
  pl.lit([list(range(n_au))]).alias("face_au"),
/var/folders/jh/mjfcxr_509gc9z3gd91thc0w0000gn/T/ipykernel_44678/2250504916.py:14: DeprecationWarning: Behavior for `lit` will change for sequence inputs. The result will change to be a literal of type List. To retain the old behavior, pass a Series instead, e.g. `Series(sequence)`.
  pl.lit([list(range(n_au))]).alias("face_au"),
/var/folders/jh/mjfcxr_509gc9z3gd91thc0w0000gn/T/ipykernel_44678/2250504916.py:14: DeprecationWarning: Behavior for `lit` will change for sequence inputs. The result will change to be a literal of type List. To retain the old behavior, pass a Series instead, e.g. `Series(sequence)`.
  pl.lit([list(range(n_au))]).alias("face_au"),
/var

In [4]:
# Comparing the formal and informal speech in terms of the mean activation of the action units

def mean_activation(df):
    mean = df.groupby('time')['value'].mean().reset_index()
    return mean

formal_mean = mean_activation(formal)
informal_mean = mean_activation(informal)

# Plotting the mean activation of the action units for formal and informal speech
fig = go.Figure()

fig.add_trace(go.Scatter(x=formal_mean['time'], y=formal_mean['value'], mode='lines', name='Formal'))
fig.add_trace(go.Scatter(x=informal_mean['time'], y=informal_mean['value'], mode='lines', name='Informal'))

# Adding traces for the dashed lines representing the averages
fig.add_trace(go.Scatter(x=formal_mean['time'], y=[formal_mean['value'].mean()] * len(formal_mean), 
                         mode='lines', name='Formal Average', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=informal_mean['time'], y=[informal_mean['value'].mean()] * len(informal_mean), 
                         mode='lines', name='Informal Average', line=dict(dash='dash')))

fig.update_layout(title='Mean Activation of Action Units for Formal and Informal Speakers',
                    xaxis_title='Time (s)',
                    yaxis_title='Mean Activation')

fig.show()

# run the T-test
from scipy.stats import ttest_ind

tstatistic, pvalue = ttest_ind(formal_mean['value'], informal_mean['value'], alternative='two-sided', nan_policy='omit')
print(f'T-statistic: {tstatistic}, P-value: {pvalue.round(5)}')

T-statistic: 16.7897650059143, P-value: 0.0


In [5]:
# comparing the mean activation of the action units between male and female speakers

male_mean = mean_activation(male)
female_mean = mean_activation(female)

fig = go.Figure()

fig.add_trace(go.Scatter(x=male_mean['time'], y=male_mean['value'], mode='lines', name='Male'))
fig.add_trace(go.Scatter(x=female_mean['time'], y=female_mean['value'], mode='lines', name='Informal'))

# Adding traces for the dashed lines representing the averages


fig.update_layout(title='Mean Activation of Action Units for Male and Female Speakers',
                    xaxis_title='Time (s)',
                    yaxis_title='Mean Activation')

fig.show()

# run the T-test
tstatistic, pvalue = ttest_ind(male_mean['value'], female_mean['value'], alternative='two-sided', nan_policy='omit')
print(f'T-statistic: {tstatistic}, P-value: {pvalue.round(5)}')

T-statistic: -3.221721682981537, P-value: 0.00133


In [6]:
# comparing the mean activation of the action units between male and female speakers

def mean_activation(df):
    mean = df.groupby('face_au')['value'].mean().reset_index()

    # calculate the confidence interval
    std = df.groupby('face_au')['value'].std().reset_index()
    mean['std'] = std['value']
    mean['upper'] = mean['value'] + mean['std']
    mean['lower'] = mean['value'] - mean['std']
    return mean

female_mean = mean_activation(female)
male_mean = mean_activation(male)


# Plotting the mean activation of the action units and the confidence interval for female speakers

# Map numerical IDs to action unit labels
au_labels = [format(au_id) for au_id in au_ref]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=au_labels,
    y=female_mean['value'],
    error_y=dict(type='data', array=female_mean['std'], visible=True),
    name='Female',
))


fig.add_trace(go.Bar(
    x=au_labels,
    y=male_mean['value'],
    error_y=dict(type='data', array=male_mean['std'], visible=True),
    name='Male',
))

# Customize the layout
fig.update_layout(
    title='Mean Activation of Action Units',
    xaxis=dict(title='Action Units', dtick=1),
    yaxis=dict(title='Mean Activation'),
    barmode='group',
    showlegend=True
)

# Show the plot
fig.show()

In [7]:
# Reference table linking emotions to action units
emotion_au_mapping = {
    'Happiness': [6, 12],
    'Sadness': [1, 4, 15],
    'Surprise': [1, 2, 5, 26],
    'Anger': [4, 5, 7, 23],
}

# Function to calculate mean activation for each emotion over time and normalize the data
def mean_activation_by_emotion_over_time(df, emotion_au_mapping):
    emotion_means_over_time = {}
    for emotion, aus in emotion_au_mapping.items():
        relevant_aus = df[df['face_au'].isin(aus)]
        emotion_means_over_time[emotion] = relevant_aus.groupby(['time', 'face_au'])['value'].mean().reset_index()

    # Normalize the data using min-max scaling for each action unit
    for emotion, mean_df in emotion_means_over_time.items():
        min_values = mean_df.groupby('face_au')['value'].min()
        max_values = mean_df.groupby('face_au')['value'].max()

        mean_df['value_normalized'] = mean_df.apply(
            lambda row: (row['value'] - min_values[row['face_au']]) / (max_values[row['face_au']] - min_values[row['face_au']]),
            axis=1
        )

    return emotion_means_over_time

# Calculate mean activation for formal and informal speakers over time and normalize the data
formal_emotion_means_over_time = mean_activation_by_emotion_over_time(female_formal_2_au, emotion_au_mapping)
informal_emotion_means_over_time = mean_activation_by_emotion_over_time(female_informal_2_au, emotion_au_mapping)

def plot_emotion_means_over_time(emotion_means_over_time, title):
    fig = go.Figure()

    for emotion, mean_df in emotion_means_over_time.items():
        for au in mean_df['face_au'].unique():
            fig.add_trace(go.Scatter(
                x=mean_df[mean_df['face_au'] == au]['time'],
                y=mean_df[mean_df['face_au'] == au]['value_normalized'],
                mode='lines+markers',
                name=f'{emotion} - AU{au}'
            ))

    fig.update_layout(
        title=f'Mean Activation of Action Units for Different Emotions Over Time (Normalized) - {title}',
        xaxis=dict(title='Time'),
        yaxis=dict(title='Normalized Mean Activation'),
        showlegend=True
    )

    fig.show()

# Plot the mean activation of action units for different emotions over time for formal speakers
plot_emotion_means_over_time(formal_emotion_means_over_time, 'Formal')
plot_emotion_means_over_time(informal_emotion_means_over_time, 'Informal')

In [8]:
# Function to calculate mean activation for each emotion, normalize the data, and count peaks
def count_peaks_by_emotion(df, emotion_au_mapping, threshold=0.85):
    emotion_counts = {emotion: 0 for emotion in emotion_au_mapping.keys()}
    
    for emotion, aus in emotion_au_mapping.items():
        relevant_aus = df[df['face_au'].isin(aus)]
        relevant_aus['value_normalized'] = (
            relevant_aus['value'] - relevant_aus['value'].min()) / (relevant_aus['value'].max() - relevant_aus['value'].min())

        # Create a new column to indicate whether the value is a peak
        relevant_aus['is_peak'] = relevant_aus['value_normalized'] > threshold

        # Count the peaks 
        peak_count = 0
        is_peak = False

        for index, row in relevant_aus.iterrows():
            if row['is_peak'] and not is_peak:
                peak_count += 1
                is_peak = True
            elif not row['is_peak']:
                is_peak = False

        # Check if the last row is a peak
        if relevant_aus.iloc[-1]['is_peak']:
            peak_count += 1

        # Update the count in the emotion_counts dictionary
        emotion_counts[emotion] = peak_count
    
    return emotion_counts

# Count peaks for formal and informal speakers
formal_peak_counts = count_peaks_by_emotion(formal, emotion_au_mapping)
informal_peak_counts = count_peaks_by_emotion(informal, emotion_au_mapping)

# Plotting the number of peaks for each emotion
fig = go.Figure()

fig.add_trace(go.Bar(
    x=list(formal_peak_counts.keys()),
    y=list(formal_peak_counts.values()),
    name='Formal',
))

fig.add_trace(go.Bar(
    x=list(informal_peak_counts.keys()),
    y=list(informal_peak_counts.values()),
    name='Informal'
))

# Customize the layout
fig.update_layout(
    title='Number of Peaks for Each Emotion',
    xaxis=dict(title='Emotion'),
    yaxis=dict(title='Number of Peaks'),
    showlegend=True
)

# Show the plot
fig.show()

/var/folders/jh/mjfcxr_509gc9z3gd91thc0w0000gn/T/ipykernel_44678/2465926288.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/jh/mjfcxr_509gc9z3gd91thc0w0000gn/T/ipykernel_44678/2465926288.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/jh/mjfcxr_509gc9z3gd91thc0w0000gn/T/ipykernel_44678/2465926288.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [9]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score


# concatenate the data
df_all = pd.concat([formal, informal])

def pivot_au(df):
    pivoted = df.pivot_table(index='frame', columns='face_au', values='value')
    return pivoted

pivoted = pivot_au(df_all)

# Apply PCA to reduce the dimensionality
pca = PCA(n_components=2)
pivoted_pca = pca.fit_transform(pivoted)

# Apply k-means clustering
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(pivoted_pca)
clusters = kmeans.predict(pivoted_pca)

# evaluate the clustering
silhouette = silhouette_score(pivoted_pca, clusters)
print(f'Silhouette score: {silhouette}')

# Plot the clusters and the labels of informal and formal speech
# with plotly go
fig = go.Figure()

fig.add_trace(go.Scatter(x=pivoted_pca[clusters==0, 0], y=pivoted_pca[clusters==0, 1], mode='markers', name='Cluster 1'))
fig.add_trace(go.Scatter(x=pivoted_pca[clusters==1, 0], y=pivoted_pca[clusters==1, 1], mode='markers', name='Cluster 2'))
fig.add_trace(go.Scatter(x=pivoted_pca[clusters==2, 0], y=pivoted_pca[clusters==2, 1], mode='markers', name='Cluster 3'))
fig.add_trace(go.Scatter(x=pivoted_pca[clusters==3, 0], y=pivoted_pca[clusters==3, 1], mode='markers', name='Cluster 4'))

fig.update_layout(title='Clusters of Formal and Informal Speech',
                    xaxis_title='PC1',
                    yaxis_title='PC2')

fig.show()

/Users/tom/anaconda3/envs/Mexca/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Silhouette score: 0.36760463694468337


In [15]:
# get the real labels
df_all['real_label'] = df_all.index.get_level_values(0)
real_labels = df_all['real_label']

